In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160612)
tf.set_random_seed(20160612)

/Users/yamamotomasaomi/.pyenv/versions/anaconda3-5.1.0/envs/makuwo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
class SingleLeyerNetWork:
    def __init__(self, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_units)
            self.prepare_session()
    
    def prepare_model(self, num_units):
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name=('input'))
            
        with tf.name_scope('hidden'):
            w1 = tf.Variable(tf.truncated_normal([784, num_units]), name='weights')
            b1 = tf.Variable(tf.zeros([num_units]),name='biases')
            hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1, name='hidden1')
        
        with tf.name_scope('output'):
            w0 = tf.Variable(tf.zeros([num_units,10]),name='weights')
            b0 = tf.Variable(tf.zeros([10]),name='biases')
            p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0, name='softmax')
            
        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32,[None, 10], name='labels')
            loss = -tf.reduce_sum(t*tf.log(p),name='loss')
            train_step = tf.train.AdamOptimizer().minimize(loss)
            
        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.arg_max(p, 1),tf.arg_max(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
            
            tf.summary.scalar('loss', loss)
            tf.summary.scalar('accuracy', accuracy)
            tf.summary.scalar('weights_hidden', w1)
            tf.summary.histogram('biases_hidden', b1)
            tf.summary.histogram('weights_output', w0)
            tf.summary.histogram('biases_output', b0)
            
            self.x, self.t, self.p = x, t, p
            self.train_step = train_step
            self.loss = loss
            self.accuracy = accuracy
            
    def prepare_session(self):
        sess = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())
        summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("/tmp/mnist_sl_logs", sess.graph)
        self.sess = sess
        self.summary = summary
        self.writer = writer

In [4]:
!rm -rf /tmp/mnist_sl/logs

In [5]:
nn = SingleLeyerNetWork(1024)

i = 0
for _ in range (2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    nn.sess.run(nn.train_step, feed_dict={nn.x:batch_xs,nn.t:batch_ts})
    if i % 100 == 0:
        summary, loss_val, acc_val = nn.sess.run([nn.summary, nn.loss, nn.accuracy], feed_dict={nn.x:mnist.test.images, nn.t:mnist.test.labels})
        print ('step: %d, loss: %f, accuracy: %f'
              %(i, loss_val, acc_val))
        nn.writer.add_summary(summary, i)
        

Instructions for updating:
Use `argmax` instead


InvalidArgumentError: tags and values not the same shape: [] != [784,1024] (tag 'evaluator/weights_hidden')
	 [[Node: evaluator/weights_hidden = ScalarSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](evaluator/weights_hidden/tags, hidden/weights/read)]]

Caused by op 'evaluator/weights_hidden', defined at:
  File "/Users/yamamotomasaomi/.pyenv/versions/anaconda3-5.1.0/envs/makuwo/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/yamamotomasaomi/.pyenv/versions/anaconda3-5.1.0/envs/makuwo/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/yamamotomasaomi/.pyenv/versions/anaconda3-5.1.0/envs/makuwo/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/yamamotomasaomi/.pyenv/versions/anaconda3-5.1.0/envs/makuwo/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start